In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from tensorflow.keras.models import load_model

In [2]:
#Load all the trained models 
model = load_model('churn_model.h5')


#Load the encoder and scaler

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('onehot_encoder_geo.pkl', 'rb') as f:
    onehot_encoder_geography = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [3]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 2,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
#Encode the data 
geo_encoded = onehot_encoder_geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geography.get_feature_names_out(['Geography']))


d:\DeepLearningProject\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [5]:
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,2,60000,2,1,1,50000


In [6]:
input_data_df = pd.concat([pd.DataFrame([input_data]), geo_encoded_df], axis=1)
input_data_df = input_data_df.drop(columns=['Geography'])

In [7]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,2,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,2,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
#Scale 
input_data_scaled = scaler.transform(input_data_df)
input_data_scaled

array([[-0.52111599,  0.91601335,  0.10961719, -1.031415  , -0.2569057 ,
         0.8095029 ,  0.64259497,  0.9687384 , -0.87203322,  0.98560362,
        -0.5698444 , -0.57369368]])

In [10]:
#predict churn 
prediction = model.predict(input_data_scaled)
prediction

1/1 [==============================] - 0s 156ms/step


array([[0.03129107]], dtype=float32)

In [11]:
if(prediction[0][0] > 0.5):
    print("Customer will churn")
else:
    print("Customer will not churn")

Customer will not churn
